In [1]:
!pip install dlib

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\lycas02\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

In [3]:
from pathlib import Path
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm
import shutil
import time
import dlib
from random import shuffle

In [4]:
detector = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")

In [5]:
videos_path = '../../raid/Data/videos/google/'

In [6]:
output_path = 'multiframe_full/'

In [7]:
for label in ['real/','fake/']:
    print(videos_path+label+': '+str(len([f for f in os.listdir(videos_path+label) if 'mp4' in f])))

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: '../../raid/Data/videos/google/real/'

In [ ]:
for label in ['real/','fake/']:
    Path(output_path+label).mkdir(parents=True, exist_ok=True)

In [ ]:
label = 'fake/'
vidpath = videos_path + label
folderpath = output_path + label
videos = sorted(os.listdir(vidpath))
shuffle(videos)
videos = videos[:len([f for f in os.listdir(videos_path+'fake/') if 'mp4' in f])]
req_frames = 10
scale = 1.3
req_size = 200

In [ ]:
for video in tqdm(videos):
    
    facepath = folderpath + video[:-4] + '/'
    Path(facepath).mkdir(parents=True,exist_ok=True)
    video_capture = cv2.VideoCapture(vidpath + video)
    count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    more_than_one = 0
    no_faces = 0
    out_of_frame = 0
    done_frames = 0
    img_size = -1
    frame_number = 0
    
    while True:
        res, frame = video_capture.read()
        if not res:
            break
        faceRects = detector(frame[:,:,::-1], 0)
        if (len(faceRects)==0):
            no_faces += 1
            continue
        if (len(faceRects)>1):
            more_than_one += 1
            continue
        x1 = faceRects[0].rect.left()
        y1 = faceRects[0].rect.top()
        x2 = faceRects[0].rect.right()
        y2 = faceRects[0].rect.bottom()
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        length = max(x2 - x1, y2 - y1)
        x1 = center[0] - int(length*scale/2)
        x2 = center[0] + int(length*scale/2)
        y1 = center[1] - int(length*scale/2)
        y2 = center[1] + int(length*scale/2)
        if x1 < 0 or y1 < 0 or x2 > frame.shape[1] or y2 > frame.shape[0]:
            out_of_frame += 1
            continue
        final=frame[y1:y2,x1:x2,::-1]
        # Either resize images now or resize them during training
        # final = cv2.resize(final,(req_size,req_size),interpolation = cv2.INTER_CUBIC)
        plt.imsave(facepath+str(frame_number).zfill(2)+'.jpg',final)
        frame_number += 1
        done_frames += 1
        if done_frames == 10:
            img_size = y2-y1
            break
    video_capture.release()
    if(len(os.listdir(facepath))==0):
        os.rmdir(facepath)
             